In [90]:
import requests
import json
import time

In [105]:
class WebScraper():

    def __init__(self, apiKey = 'd5a45936c03454b7e98aa0a4f9e45e4a', enable_dollar_conversion = True) -> None:
        
        self.apiKey = apiKey
        self.country_to_ext = {"malasya": "my", "singapore": "sg", "vietnam": "vn", "thailand": "th", "indonesia": "id"}
        self.country_to_curr = {"Malasya": "MYR", "Singapore": "SGD", "Vietnam": "VND", "Thailand": "THB", "Indonesia": "IDR"}
        self.enable_dollar_conversion = enable_dollar_conversion
        if self.enable_dollar_conversion:
            with open("./utils/exchange_rates.json", "r") as f:
                self.conversion_rates = json.load(f)

    @staticmethod
    def all_finished(statuses):

        return statuses == ['finished']*len(statuses)

    @staticmethod
    def find_ind_not_finished(statuses):

        inds = []
        for i, status in enumerate(statuses):
            if status != 'finished':
                inds.append(i)
        return inds

    def update_conversion_rate(self):
        base_url = "https://fxds-public-exchange-rates-api.oanda.com/cc-api/currencies?base=[CURR]&quote=USD&data_type=general_currency_pair&start_date=2023-02-12&end_date=2023-02-13"
        r = requests.post(url = 'https://async.scraperapi.com/batchjobs', json={ 'apiKey': self.apiKey, 'urls': [base_url.replace("[CURR]", curr) for curr in self.country_to_curr.values()]})
        time.sleep(3)
        responses = [requests.get(url = res["statusUrl"]) for res in r.json()]
        statuses = [resp.json()["status"] for resp in responses]
        i = 0
        while i < 10 and not WebScraper.all_finished(statuses):
            i += 1
            time.sleep(2)
            inds_to_rerun = WebScraper.find_ind_not_finished(statuses)
            for ind in inds_to_rerun:
                responses[ind] = requests.get(url = r.json()[ind]["statusUrl"])
            statuses = [resp.json()["status"] for resp in responses]
        assert(i != 10), "oops, it doesn't seem to work for now, please try again later."
        values = [resp.json()["response"]["body"]["response"][0]["average_bid"] for resp in responses]
        self.conversion_rates = {key: value for key, value in zip(self.country_to_curr.keys(), values)}
        with open("./utils/exchange_rates.json", "w") as f:
            json.dump(self.conversion_rates, f)

    def get_seller_data(self, shopid, country, filter_sold_out=0, chunk_size=100):
        
        assert(country.lower() in ["malasya", "singapore", "vietnam", "thailand", "indonesia"]), 'The provided country should be one of the following countries : "Malasya", "Singapore", "Vietnam", "Thailand" or "Indonesia".'
        assert(type(shopid)==int), f"The shopid should be an integer. you provided a value of type {type(shopid)}."

        base_url = f'https://shopee.{self.country_to_ext[country.lower()]}/api/v4/shop/search_items?filter_sold_out={filter_sold_out}&limit={chunk_size}&offset=0&order=desc&shopid={shopid}&sort_by=pop&use_case=4'

        r = requests.post(url = 'https://async.scraperapi.com/jobs', json={ 'apiKey': self.apiKey, 'url': base_url})
        time.sleep(3)
        response = requests.get(url = r.json()["statusUrl"])
        status = response.json()["status"]
        i = 0
        while i < 10 and status != 'finished':
            i += 1
            time.sleep(2)
            response = requests.get(url = r.json()["statusUrl"])
            status = response.json()["status"]
        return response

In [106]:
ws = WebScraper()

In [107]:
resp = ws.get_seller_data(shopid=33697514, country="indonesia")

In [110]:
resp.json()

{'id': '88a28d16-6c4c-4fe7-a5e3-ca257d8ba8d9',
 'status': 'running',
 'statusUrl': 'https://async.scraperapi.com/jobs/88a28d16-6c4c-4fe7-a5e3-ca257d8ba8d9',
 'url': 'https://shopee.id/api/v4/shop/search_items?filter_sold_out=0&limit=100&offset=0&order=desc&shopid=33697514&sort_by=pop&use_case=4'}